## Traffic tensor completion using truncated sparsity-inducing regularizer
- Main functions: LATC_HOC (THOC), LATC_HOP (THOP)
- Refer to Chen et al. (2021) for LATC-related method (link: https://github.com/xinychen/transdim/blob/master/imputer/LATC.ipynb)

## Functions for implementation

In [28]:
import numpy as np
import time

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, tensor_size, mode):
    index = list()
    index.append(mode)
    for i in range(tensor_size.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, tensor_size[index].tolist(), order = 'F'), 0, mode)

def svt_tnn(mat, tau, theta):
    [m, n] = mat.shape
    if 2 * m < n:
        u, s, v = np.linalg.svd(mat @ mat.T, full_matrices = 0)
        s = np.sqrt(s)
        idx = np.sum(s > tau)
        mid = np.zeros(idx)
        mid[: theta] = 1
        mid[theta : idx] = (s[theta : idx] - tau) / s[theta : idx]
        return (u[:, : idx] @ np.diag(mid)) @ (u[:, : idx].T @ mat)
    elif m > 2 * n:
        return svt_tnn(mat.T, tau, theta).T
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    idx = np.sum(s > tau)
    vec = s[: idx].copy()
    vec[theta : idx] = s[theta : idx] - tau
    return u[:, : idx] @ np.diag(vec) @ v[: idx, :]

def svd_(mat):
    # faster SVD
    [m, n] = mat.shape
    try:
        if 2 * m < n:
            u, s, _ = np.linalg.svd(mat @ mat.T, full_matrices=False)
            s = np.sqrt(s)
            tol = n * torch.finfo(float).eps
            idx = np.sum(s > tol)
            return u[:, :idx], s[:idx],  np.diag(1/s[:idx]) @ u[:, :idx].T @ mat
        elif m > 2 * n:
            v, s, u = svd_(mat.T)
            return u, s, v
    except: 
        pass
    u, s, v = np.linalg.svd(mat, full_matrices=False)
    return u, s, v

def ReLU(x):
    return x * (x > 0)

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

from scipy import sparse
from scipy.sparse.linalg import spsolve as spsolve

def generate_Psi(dim_time, time_lags):
    Psis = []
    max_lag = np.max(time_lags)
    for i in range(len(time_lags) + 1):
        row = np.arange(0, dim_time - max_lag)
        if i == 0:
            col = np.arange(0, dim_time - max_lag) + max_lag
        else:
            col = np.arange(0, dim_time - max_lag) + max_lag - time_lags[i - 1]
        data = np.ones(dim_time - max_lag)
        Psi = sparse.coo_matrix((data, (row, col)), shape = (dim_time - max_lag, dim_time))
        Psis.append(Psi)
    return Psis

## LATC 

In [31]:
def latc(dense_tensor, sparse_tensor, time_lags, alpha, rho0, lambda0, theta, 
         epsilon = 1e-4, maxiter = 100, K = 3):
    """Low-Rank Autoregressive Tensor Completion (LATC)"""
    
    dim = np.array(sparse_tensor.shape)
    dim_time = int(np.prod(dim) / dim[0])
    d = len(time_lags)
    max_lag = np.max(time_lags)
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    dense_test = dense_tensor[pos_test]
    del dense_tensor
    
    # Initialize T, Z, A
    T = np.zeros(dim)
    Z_tensor = sparse_tensor.copy()
    Z = sparse_mat.copy()
    A = 0.001 * np.random.rand(dim[0], d)
    Psis = generate_Psi(dim_time, time_lags)
    iden = sparse.coo_matrix((np.ones(dim_time), (np.arange(0, dim_time), np.arange(0, dim_time))), 
                             shape = (dim_time, dim_time))
    it = 0
    ind = np.zeros((d, dim_time - max_lag), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(max_lag - time_lags[i], dim_time - time_lags[i])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    rho = rho0
    
    used_time = 0
    
    # Main loop
    while True:
        start_time = time.time()
        
        temp = []
        for m in range(dim[0]):
            Psis0 = Psis.copy()
            for i in range(d):
                Psis0[i + 1] = A[m, i] * Psis[i + 1]
            B = Psis0[0] - sum(Psis0[1 :])
            temp.append(B.T @ B)
            
        # loop for ADMM
        for k in range(K):
            rho = min(rho * 1.05, 1e5)
            tensor_hat = np.zeros(dim)
            
            # Update X
            for p in range(len(dim)):
                tensor_hat += alpha[p] * mat2ten(svt_tnn(ten2mat(Z_tensor - T / rho, p), 
                                                         alpha[p] / rho, theta), dim, p)
                
            # Update Z
            temp0 = rho / lambda0 * ten2mat(tensor_hat + T / rho, 0)
            mat = np.zeros((dim[0], dim_time))
            for m in range(dim[0]):
                mat[m, :] = spsolve(temp[m] + rho * iden / lambda0, temp0[m, :])
            Z[pos_missing] = mat[pos_missing]
            Z_tensor = mat2ten(Z, dim, 0)
            
            # Update T
            T = T + rho * (tensor_hat - Z_tensor)
        
        # Update A
        for m in range(dim[0]):
            A[m, :] = np.linalg.lstsq(Z[m, ind].T, Z[m, max_lag :], rcond = None)[0]
        mat_hat = ten2mat(tensor_hat, 0)
        
        
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        if it % 20 == 0:
            print("iter", it, "tole = %.5f" % tol)
            #print_result(it, tol, dense_test, tensor_hat[pos_test])
            
        used_time += time.time() - start_time
        
        if (tol < epsilon) or (it >= maxiter):
            break
    #print_result(it, tol, dense_test, tensor_hat[pos_test])
    mape = compute_mape(dense_test, tensor_hat[pos_test])
    rmse = compute_rmse(dense_test, tensor_hat[pos_test])
    
    return tensor_hat, used_time, mape, rmse

## LATC+HOC

In [34]:
def hoc_shrinkage(vec, lam, gamma):
    tmp = np.abs(vec) - (gamma**2 + lam**2)*np.abs(vec)/(gamma**2+vec**2)
    return np.sign(vec)*ReLU(tmp)

def svt_hoc(mat, lam, gamma, theta=0.1):    
    u, s, v = svd_(mat)
    ss = s.copy()
    r = int(np.ceil(theta * len(s)))
    #r = 15
    ss[r:] = hoc_shrinkage(s[r:], lam, gamma)
    idx = np.where(ss > 0)[0]
    return u[:, idx] @ np.diag(ss[idx]) @ v[idx, :]


In [36]:
def latc_hoc(dense_tensor, sparse_tensor, time_lags, alpha, rho0, lambda0, theta, rhofac,
             epsilon = 1e-4, maxiter = 200, K = 3):
    """ Truncated hybrid-ordinary Cauchy (THOC)"""
    
    dim = np.array(sparse_tensor.shape)
    dim_time = int(np.prod(dim) / dim[0])
    d = len(time_lags)
    max_lag = np.max(time_lags)
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    dense_test = dense_tensor[pos_test]
    del dense_tensor
    
    # Initialize T, Z, A
    T = np.zeros(dim)
    Z_tensor = sparse_tensor.copy()
    Z = sparse_mat.copy()
    A = 0.001 * np.random.rand(dim[0], d)
    Psis = generate_Psi(dim_time, time_lags)
    iden = sparse.coo_matrix((np.ones(dim_time), (np.arange(0, dim_time), np.arange(0, dim_time))), 
                             shape = (dim_time, dim_time))
    it = 0
    ind = np.zeros((d, dim_time - max_lag), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(max_lag - time_lags[i], dim_time - time_lags[i])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    rho = rho0
    
    used_time = 0
    sigmafac = np.sqrt(2)
    
    # Main loop
    while True:
        start_time = time.time()
        
        temp = []
        for m in range(dim[0]):
            Psis0 = Psis.copy()
            for i in range(d):
                Psis0[i + 1] = A[m, i] * Psis[i + 1]
            B = Psis0[0] - sum(Psis0[1 :])
            temp.append(B.T @ B)
            
        # loop for ADMM
        for k in range(K):
            rho = min(rho * rhofac, 1e5)
            tensor_hat = np.zeros(dim)
            
            # Update X
            for p in range(len(dim)):
                lam = alpha[p]/rho
                gamma = lam
                tmp_p = svt_hoc(ten2mat(Z_tensor- T/rho, p), lam, gamma, theta)
                tensor_hat += alpha[p]*mat2ten(tmp_p, dim, p)
                #tensor_hat += alpha[p] * mat2ten(svt_tnn(ten2mat(Z_tensor - T / rho, p), 
                #                                         alpha[p] / rho, theta), dim, p)
                
            # Update Z
            temp0 = rho / lambda0 * ten2mat(tensor_hat + T / rho, 0)
            mat = np.zeros((dim[0], dim_time))
            for m in range(dim[0]):
                mat[m, :] = spsolve(temp[m] + rho * iden / lambda0, temp0[m, :])
            Z[pos_missing] = mat[pos_missing]
            Z_tensor = mat2ten(Z, dim, 0)
            
            # Update T
            T = T + rho * (tensor_hat - Z_tensor)
        
        # Update A
        for m in range(dim[0]):
            A[m, :] = np.linalg.lstsq(Z[m, ind].T, Z[m, max_lag :], rcond = None)[0]
        mat_hat = ten2mat(tensor_hat, 0)
        
        
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        if it % 20 == 0:
            print("iter", it, "tole = %.5f" % tol)
            #print_result(it, tol, dense_test, tensor_hat[pos_test])
            
        used_time += time.time() - start_time
        
        if (tol < epsilon) or (it >= maxiter):
            break
    #print_result(it, tol, dense_test, tensor_hat[pos_test])
    mape = compute_mape(dense_test, tensor_hat[pos_test])
    rmse = compute_rmse(dense_test, tensor_hat[pos_test])
    
    return tensor_hat, used_time, mape, rmse

## LATC+HOP

In [39]:
## THOP
def hop_shrinkage(vec, lam, p):
    tmp = np.abs(vec)-(lam**(2-p))*(np.abs(vec)**(p-1))
    return np.sign(vec)*ReLU(tmp)

def svt_hop(mat, lam, p, theta=0.1):    
    u, s, v = svd_(mat)
    ss = s.copy()
    r = int(np.ceil(theta * len(s)))
    #r = 15
    ss[r:] = hop_shrinkage(s[r:], lam, p)
    idx = np.where(ss > 0)[0]
    return u[:, idx] @ np.diag(ss[idx]) @ v[idx, :]

In [51]:
def latc_hop(dense_tensor, sparse_tensor, time_lags, alpha, rho0, lambda0, theta, rhofac, p_hop,
             epsilon = 1e-4, maxiter = 200, K = 3):
    """ Truncated hybrid-ordinary Cauchy (THOC) """
    
    dim = np.array(sparse_tensor.shape)
    dim_time = int(np.prod(dim) / dim[0])
    d = len(time_lags)
    max_lag = np.max(time_lags)
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    dense_test = dense_tensor[pos_test]
    del dense_tensor
    
    # Initialize T, Z, A
    T = np.zeros(dim)
    Z_tensor = sparse_tensor.copy()
    Z = sparse_mat.copy()
    A = 0.001 * np.random.rand(dim[0], d)
    Psis = generate_Psi(dim_time, time_lags)
    iden = sparse.coo_matrix((np.ones(dim_time), (np.arange(0, dim_time), np.arange(0, dim_time))), 
                             shape = (dim_time, dim_time))
    it = 0
    ind = np.zeros((d, dim_time - max_lag), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(max_lag - time_lags[i], dim_time - time_lags[i])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    rho = rho0
    
    used_time = 0
    sigmafac = np.sqrt(2)
    #p_hop = 0.1
    
    # Main loop
    while True:
        start_time = time.time()
        
        temp = []
        for m in range(dim[0]):
            Psis0 = Psis.copy()
            for i in range(d):
                Psis0[i + 1] = A[m, i] * Psis[i + 1]
            B = Psis0[0] - sum(Psis0[1 :])
            temp.append(B.T @ B)
            
        # loop for ADMM
        for k in range(K):
            rho = min(rho * rhofac, 1e5)
            tensor_hat = np.zeros(dim)
            
            # Update X
            for p in range(len(dim)):
                lam = alpha[p]/rho
                gamma = lam
                tmp_p = svt_hop(ten2mat(Z_tensor- T/rho, p), lam, p_hop, theta)
                tensor_hat += alpha[p]*mat2ten(tmp_p, dim, p)
                #tensor_hat += alpha[p] * mat2ten(svt_tnn(ten2mat(Z_tensor - T / rho, p), 
                #                                         alpha[p] / rho, theta), dim, p)
                
            # Update Z
            temp0 = rho / lambda0 * ten2mat(tensor_hat + T / rho, 0)
            mat = np.zeros((dim[0], dim_time))
            for m in range(dim[0]):
                mat[m, :] = spsolve(temp[m] + rho * iden / lambda0, temp0[m, :])
            Z[pos_missing] = mat[pos_missing]
            Z_tensor = mat2ten(Z, dim, 0)
            
            # Update T
            T = T + rho * (tensor_hat - Z_tensor)
        
        # Update A
        for m in range(dim[0]):
            A[m, :] = np.linalg.lstsq(Z[m, ind].T, Z[m, max_lag :], rcond = None)[0]
        mat_hat = ten2mat(tensor_hat, 0)
        
        
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        if it % 20 == 0:
            print("iter", it, "tole = %.5f" % tol)
            #print_result(it, tol, dense_test, tensor_hat[pos_test])
            
        used_time += time.time() - start_time
        
        if (tol < epsilon) or (it >= maxiter):
            break
    #print_result(it, tol, dense_test, tensor_hat[pos_test])
    mape = compute_mape(dense_test, tensor_hat[pos_test])
    rmse = compute_rmse(dense_test, tensor_hat[pos_test])
    
    return tensor_hat, used_time, mape, rmse

## PEMv (traffic volume) dataset

In [53]:
import time
import numpy as np
import scipy.io


## Random Missing (RM)
#dense_tensor = scipy.io.loadmat('huangzhou.mat')['tensor'].transpose(0, 2, 1)

tensor = scipy.io.loadmat('dataset/PEMS08/pems08_288_62_170.mat')
dense_tensor = tensor['T']
dense_tensor = dense_tensor.transpose(2,0,1)
#dense_tensor = scipy.io.loadmat('PeMS-dataset/PEMS08/pems08_288_62_170.mat')
dense_tensor = dense_tensor[:,:,0:31]
dim1, dim2, dim3 = dense_tensor.shape
dim = dense_tensor.shape

print(dim)

(170, 288, 31)


## Simulation code ##

### RM 30%

In [40]:
import time
import numpy as np
import scipy.io

time_lags = np.arange(1,7)
alpha = np.ones(3) / 3

#### Main simulations ####

#mape5_latc = []
#rmse5_latc = []

mape5_hoc = []
rmse5_hoc = []

mape5_hop = []
rmse5_hop = []

# missing rates and pattern
missing_rate = 0.3
missing_case = "RM"

print("start")
print("missing pattern", missing_case, "missing rate =", missing_rate)

start_time = time.time()
for seed in range(1,6):
    ### generate missing entries ###
    np.random.seed(seed+999)
    
    if missing_case == 'RM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)
        #alpha = [0.1,0.8,0.1]
    elif missing_case == 'NM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[2]) + 0.5 - missing_rate)[:, None, :]
        #alpha = np.ones(3)/3
    else:
        dim_time = dim2 * dim3
        block_window = 4
        vec = np.random.rand(int(dim_time / block_window))
        temp = np.array([vec] * block_window)
        vec = temp.reshape([dim2 * dim3], order = 'F')
        sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], 
                                np.array([dim1, dim2, dim3]), 0)
        #alpha = np.ones(3)/3

    print("lag", time_lags)

    ### LATC ###
    #print("LATC", seed)
    #c = 10
    #theta = 25
    #rho = 1e-5
    #lambda0 = c * rho

    #tensor_hat, time_latc, mape_latc, rmse_latc = latc(dense_tensor, sparse_tensor, time_lags, 
    #                                                   alpha, rho, lambda0, theta)
    #print("MAPE:", mape_latc*100)
    #print("RMAE:", rmse_latc)
    #mape5_latc.append(mape_latc)
    #rmse5_latc.append(rmse_latc)

    ### THOC ###
    #print("THOC", seed)
    c = 10
    rho = 1e-5
    lambda0 = c * rho
    theta_hoc = 0.05
    
    rhofac = 1.05
    Xest, time_hoc, mape_hoc, rmse_hoc = latc_hoc(dense_tensor, sparse_tensor, time_lags, 
                                                  alpha, rho, lambda0, theta_hoc, rhofac, epsilon=1e-4)
    print("MAPE:", mape_hoc*100)
    print("RMAE:", rmse_hoc)
    mape5_hoc.append(mape_hoc)
    rmse5_hoc.append(rmse_hoc)
    
    ### LATC-HOP  ###
    #print("THOP", seed)
    c = 10
    rho = 1e-5
    lambda0 = c * rho
    theta_hop = 0.05
    p_hop = 0.5
    
    rhofac = 1.05
    Xest, time_hop, mape_hop, rmse_hop = latc_hop(dense_tensor, sparse_tensor, time_lags, 
                                                   alpha, rho, lambda0, theta_hop, rhofac, p_hop, epsilon=1e-4)
    print("mape", mape_hop*100)
    print("rmse", rmse_hop)
    mape5_hop3.append(mape_hop)
    rmse5_hop3.append(rmse_hop)

end_time = time.time()
print(end_time-start_time)

start
missing pattern RM missing rate = 0.3
lag [1 2 3 4 5 6]
iter 20 tole = 0.00215
iter 40 tole = 0.00029
mape 7.910571081913188
rmse 19.88956693233205
lag [1 2 3 4 5 6]
iter 20 tole = 0.00211
iter 40 tole = 0.00029
mape 8.002325583525
rmse 19.97249495351229
lag [1 2 3 4 5 6]
iter 20 tole = 0.00240
iter 40 tole = 0.00029
mape 7.958033092716965
rmse 19.965672772973182
lag [1 2 3 4 5 6]
iter 20 tole = 0.00220
iter 40 tole = 0.00030
mape 7.906474147855737
rmse 19.93925264684549
lag [1 2 3 4 5 6]
iter 20 tole = 0.00211
iter 40 tole = 0.00029
mape 7.965203844403189
rmse 19.920954970402242
1815.1793808937073


In [41]:
print("MAPE(%) comparison")
#print("LATC:", format(np.mean(mape5_latc)*100,".2f"))
print("LATC-HOC:", format(np.mean(mape5_hoc)*100,".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(mape5_hop)*100,".2f"))
print("--------")
print("RMSE comparison")
#print("LATC:", format(np.mean(rmse5_latc),".2f"))
print("LATC-HOC:", format(np.mean(rmse5_hoc),".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(rmse5_hop),".2f"))

MAPE(%) comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 7.95
LATC-HOP(p=0.6): nan
--------
RMSE comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 19.94
LATC-HOP(p=0.6): nan


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


### RM 70%

In [44]:
import time
import numpy as np
import scipy.io

time_lags = np.arange(1,7)
alpha = np.ones(3) / 3

#### Main simulations ####

#mape5_latc = []
#rmse5_latc = []

mape5_hoc = []
rmse5_hoc = []

mape5_hop = []
rmse5_hop = []

# missing rates and pattern
missing_rate = 0.7
missing_case = "RM"

print("start")
print("missing pattern", missing_case, "missing rate =", missing_rate)

start_time = time.time()
for seed in range(1,6):
    ### generate missing entries ###
    np.random.seed(seed+999)
    
    if missing_case == 'RM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)
        #alpha = [0.1,0.8,0.1]
    elif missing_case == 'NM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[2]) + 0.5 - missing_rate)[:, None, :]
        #alpha = np.ones(3)/3
    else:
        dim_time = dim2 * dim3
        block_window = 4
        vec = np.random.rand(int(dim_time / block_window))
        temp = np.array([vec] * block_window)
        vec = temp.reshape([dim2 * dim3], order = 'F')
        sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], 
                                np.array([dim1, dim2, dim3]), 0)
        #alpha = np.ones(3)/3

    print("lag", time_lags)

    ### LATC ###
    #print("LATC", seed)
    #c = 10
    #theta = 25
    #rho = 1e-5
    #lambda0 = c * rho

    #tensor_hat, time_latc, mape_latc, rmse_latc = latc(dense_tensor, sparse_tensor, time_lags, 
    #                                                   alpha, rho, lambda0, theta)
    #print("MAPE:", mape_latc*100)
    #print("RMAE:", rmse_latc)
    #mape5_latc.append(mape_latc)
    #rmse5_latc.append(rmse_latc)

    ### THOC ###
    #print("THOC", seed)
    c = 10
    rho = 1e-5
    lambda0 = c * rho
    theta_hoc = 0.03
    
    rhofac = 1.05
    Xest, time_hoc, mape_hoc, rmse_hoc = latc_hoc(dense_tensor, sparse_tensor, time_lags, 
                                                  alpha, rho, lambda0, theta_hoc, rhofac, epsilon=1e-4)
    print("MAPE:", mape_hoc*100)
    print("RMAE:", rmse_hoc)
    mape5_hoc.append(mape_hoc)
    rmse5_hoc.append(rmse_hoc)
    
    ### LATC-HOP  ###
    #print("THOP", seed)
    c = 10
    rho = 1e-5
    lambda0 = c * rho
    theta_hop = 0.03
    p_hop = 0.5
    
    rhofac = 1.05
    Xest, time_hop, mape_hop, rmse_hop = latc_hop(dense_tensor, sparse_tensor, time_lags, 
                                                   alpha, rho, lambda0, theta_hop, rhofac, p_hop, epsilon=1e-4)
    print("mape", mape_hop*100)
    print("rmse", rmse_hop)
    mape5_hop3.append(mape_hop)
    rmse5_hop3.append(rmse_hop)

end_time = time.time()
print(end_time-start_time)

start
missing pattern RM missing rate = 0.7
lag [1 2 3 4 5 6]
iter 20 tole = 0.00597
iter 40 tole = 0.00117
iter 60 tole = 0.00012
lag [1 2 3 4 5 6]
iter 20 tole = 0.00599
iter 40 tole = 0.00115
iter 60 tole = 0.00012
lag [1 2 3 4 5 6]
iter 20 tole = 0.00601
iter 40 tole = 0.00121
iter 60 tole = 0.00012
lag [1 2 3 4 5 6]
iter 20 tole = 0.00600
iter 40 tole = 0.00121
iter 60 tole = 0.00013
lag [1 2 3 4 5 6]
iter 20 tole = 0.00600
iter 40 tole = 0.00119
iter 60 tole = 0.00012
2305.9361000061035


In [48]:
print("MAPE(%) comparison")
#print("LATC:", format(np.mean(mape5_latc)*100,".2f"))
print("LATC-HOC:", format(np.mean(mape5_hoc)*100,".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(mape5_hop)*100,".2f"))
print("--------")
print("RMSE comparison")
#print("LATC:", format(np.mean(rmse5_latc),".2f"))
print("LATC-HOC:", format(np.mean(rmse5_hoc),".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(rmse5_hop),".2f"))

MAPE(%) comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 8.89
LATC-HOP(p=0.6): nan
--------
RMSE comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 21.98
LATC-HOP(p=0.6): nan


### NM 30%

In [77]:
import time
import numpy as np
import scipy.io

time_lags = np.arange(1,7)
alpha = np.ones(3) / 3

#### Main simulations ####

#mape5_latc = []
#rmse5_latc = []

mape5_hoc = []
rmse5_hoc = []

mape5_hop = []
rmse5_hop = []

# missing rates and pattern
missing_rate = 0.3
missing_case = "NM"

print("start")
print("missing pattern", missing_case, "missing rate =", missing_rate)

start_time = time.time()
for seed in range(1,6):
    ### generate missing entries ###
    np.random.seed(seed+999)
    
    if missing_case == 'RM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)
        #alpha = [0.1,0.8,0.1]
    elif missing_case == 'NM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[2]) + 0.5 - missing_rate)[:, None, :]
        #alpha = np.ones(3)/3
    else:
        dim_time = dim2 * dim3
        block_window = 4
        vec = np.random.rand(int(dim_time / block_window))
        temp = np.array([vec] * block_window)
        vec = temp.reshape([dim2 * dim3], order = 'F')
        sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], 
                                np.array([dim1, dim2, dim3]), 0)
        #alpha = np.ones(3)/3

    print("lag", time_lags)

    ### LATC ###
    #print("LATC", seed)
    #c = 10
    #theta = 25
    #rho = 1e-5
    #lambda0 = c * rho

    #tensor_hat, time_latc, mape_latc, rmse_latc = latc(dense_tensor, sparse_tensor, time_lags, 
    #                                                   alpha, rho, lambda0, theta)
    #print("MAPE:", mape_latc*100)
    #print("RMAE:", rmse_latc)
    #mape5_latc.append(mape_latc)
    #rmse5_latc.append(rmse_latc)

    ### THOC ###
    #print("THOC", seed)
    c = 0.1
    rho = 1e-5
    lambda0 = c * rho
    theta_hoc = 0.01
    
    rhofac = 1.05
    Xest, time_hoc, mape_hoc, rmse_hoc = latc_hoc(dense_tensor, sparse_tensor, time_lags, 
                                                  alpha, rho, lambda0, theta_hoc, rhofac, epsilon=1e-4)
    print("MAPE:", mape_hoc*100)
    print("RMAE:", rmse_hoc)
    mape5_hoc.append(mape_hoc)
    rmse5_hoc.append(rmse_hoc)
    
    ### LATC-HOP  ###
    #print("THOP", seed)
    c = 1
    rho = 1e-5
    lambda0 = c * rho
    theta_hop = 0.05
    p_hop = 0.5
    
    rhofac = 1.05
    Xest, time_hop, mape_hop, rmse_hop = latc_hop(dense_tensor, sparse_tensor, time_lags, 
                                                   alpha, rho, lambda0, theta_hop, rhofac, p_hop, epsilon=1e-4)
    print("mape", mape_hop*100)
    print("rmse", rmse_hop)
    mape5_hop3.append(mape_hop)
    rmse5_hop3.append(rmse_hop)

end_time = time.time()
print(end_time-start_time)

start
missing pattern NM missing rate = 0.3
lag [1 2 3 4 5 6]
LATC-HOP3 1
iter 20 tole = 0.00120
iter 40 tole = 0.00009
MAPE 11.78002054272138
RMSE 32.469032365247166
lag [1 2 3 4 5 6]
LATC-HOP3 2
iter 20 tole = 0.00117
iter 40 tole = 0.00010
MAPE 11.977292916254036
RMSE 31.27433197739748
lag [1 2 3 4 5 6]
LATC-HOP3 3
iter 20 tole = 0.00112
iter 40 tole = 0.00010
MAPE 12.522608563142565
RMSE 32.4036565849305
lag [1 2 3 4 5 6]
LATC-HOP3 4
iter 20 tole = 0.00118
iter 40 tole = 0.00010
MAPE 12.103894162546535
RMSE 32.78913851685067
lag [1 2 3 4 5 6]
LATC-HOP3 5
iter 20 tole = 0.00114
iter 40 tole = 0.00010
MAPE 11.055650945645311
RMSE 31.843427733604845
1368.641098022461


In [81]:
print("MAPE(%) comparison")
#print("LATC:", format(np.mean(mape5_latc)*100,".2f"))
print("LATC-HOC:", format(np.mean(mape5_hoc)*100,".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(mape5_hop)*100,".2f"))
print("--------")
print("RMSE comparison")
#print("LATC:", format(np.mean(rmse5_latc),".2f"))
print("LATC-HOC:", format(np.mean(rmse5_hoc),".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(rmse5_hop),".2f"))

MAPE(%) comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 11.89
LATC-HOP(p=0.6): nan
--------
RMSE comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 32.16
LATC-HOP(p=0.6): nan


### NM 70%

In [49]:
import time
import numpy as np
import scipy.io

time_lags = np.arange(1,7)
alpha = np.ones(3) / 3

#### Main simulations ####

#mape5_latc = []
#rmse5_latc = []

mape5_hoc = []
rmse5_hoc = []

mape5_hop = []
rmse5_hop = []

# missing rates and pattern
missing_rate = 0.7
missing_case = "NM"

print("start")
print("missing pattern", missing_case, "missing rate =", missing_rate)

start_time = time.time()
for seed in range(1,6):
    ### generate missing entries ###
    np.random.seed(seed+999)
    
    if missing_case == 'RM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)
        #alpha = [0.1,0.8,0.1]
    elif missing_case == 'NM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[2]) + 0.5 - missing_rate)[:, None, :]
        #alpha = np.ones(3)/3
    else:
        dim_time = dim2 * dim3
        block_window = 4
        vec = np.random.rand(int(dim_time / block_window))
        temp = np.array([vec] * block_window)
        vec = temp.reshape([dim2 * dim3], order = 'F')
        sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], 
                                np.array([dim1, dim2, dim3]), 0)
        #alpha = np.ones(3)/3

    print("lag", time_lags)

    ### LATC ###
    #print("LATC", seed)
    #c = 10
    #theta = 25
    #rho = 1e-5
    #lambda0 = c * rho

    #tensor_hat, time_latc, mape_latc, rmse_latc = latc(dense_tensor, sparse_tensor, time_lags, 
    #                                                   alpha, rho, lambda0, theta)
    #print("MAPE:", mape_latc*100)
    #print("RMAE:", rmse_latc)
    #mape5_latc.append(mape_latc)
    #rmse5_latc.append(rmse_latc)

    ### THOC ###
    #print("THOC", seed)
    c = 0.1
    rho = 1e-5
    lambda0 = c * rho
    theta_hoc = 0.01
    
    rhofac = 1.05
    Xest, time_hoc, mape_hoc, rmse_hoc = latc_hoc(dense_tensor, sparse_tensor, time_lags, 
                                                  alpha, rho, lambda0, theta_hoc, rhofac, epsilon=1e-4)
    print("MAPE:", mape_hoc*100)
    print("RMAE:", rmse_hoc)
    mape5_hoc.append(mape_hoc)
    rmse5_hoc.append(rmse_hoc)
    
    ### LATC-HOP  ###
    #print("THOP", seed)
    c = 1
    rho = 1e-5
    lambda0 = c * rho
    theta_hop = 0.01
    p_hop = 0.5
    
    rhofac = 1.05
    Xest, time_hop, mape_hop, rmse_hop = latc_hop(dense_tensor, sparse_tensor, time_lags, 
                                                   alpha, rho, lambda0, theta_hop, rhofac, p_hop, epsilon=1e-4)
    print("mape", mape_hop*100)
    print("rmse", rmse_hop)
    mape5_hop3.append(mape_hop)
    rmse5_hop3.append(rmse_hop)

end_time = time.time()
print(end_time-start_time)

start
missing pattern NM missing rate = 0.7
lags [1 2 3 4 5 6]
LATC-HOC 1
iter 20 tole = 0.00405
iter 40 tole = 0.00046
mape: 13.979643099389211
rmse 36.796417716720946
lags [1 2 3 4 5 6]
LATC-HOC 2
iter 20 tole = 0.00387
iter 40 tole = 0.00047
mape: 14.000163445252472
rmse 35.51087341858869
lags [1 2 3 4 5 6]
LATC-HOC 3
iter 20 tole = 0.00403
iter 40 tole = 0.00044
mape: 14.113695465976514
rmse 36.86434662628388
lags [1 2 3 4 5 6]
LATC-HOC 4
iter 20 tole = 0.00395
iter 40 tole = 0.00049
mape: 13.9676275625115
rmse 35.39644715460936
lags [1 2 3 4 5 6]
LATC-HOC 5
iter 20 tole = 0.00394
iter 40 tole = 0.00051
mape: 13.940535727259789
rmse 36.55228347300118
1718.3546681404114


In [53]:
print("MAPE(%) comparison")
#print("LATC:", format(np.mean(mape5_latc)*100,".2f"))
print("LATC-HOC:", format(np.mean(mape5_hoc)*100,".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(mape5_hop)*100,".2f"))
print("--------")
print("RMSE comparison")
#print("LATC:", format(np.mean(rmse5_latc),".2f"))
print("LATC-HOC:", format(np.mean(rmse5_hoc),".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(rmse5_hop),".2f"))

MAPE(%) comparison
LATC: nan
LATC-HOC: 14.00
LATC-HOP(p=0.3): nan
LATC-HOP(p=0.6): nan
--------
RMSE comparison
LATC: nan
LATC-HOC: 36.22
LATC-HOP(p=0.3): nan
LATC-HOP(p=0.6): nan


### BM 30%

In [63]:
import time
import numpy as np
import scipy.io

time_lags = np.arange(1,7)
alpha = np.ones(3) / 3

#### Main simulations ####

#mape5_latc = []
#rmse5_latc = []

mape5_hoc = []
rmse5_hoc = []

mape5_hop = []
rmse5_hop = []

# missing rates and pattern
missing_rate = 0.7
missing_case = "NM"

print("start")
print("missing pattern", missing_case, "missing rate =", missing_rate)

start_time = time.time()
for seed in range(1,6):
    ### generate missing entries ###
    np.random.seed(seed+999)
    
    if missing_case == 'RM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)
        #alpha = [0.1,0.8,0.1]
    elif missing_case == 'NM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[2]) + 0.5 - missing_rate)[:, None, :]
        #alpha = np.ones(3)/3
    else:
        dim_time = dim2 * dim3
        block_window = 4
        vec = np.random.rand(int(dim_time / block_window))
        temp = np.array([vec] * block_window)
        vec = temp.reshape([dim2 * dim3], order = 'F')
        sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], 
                                np.array([dim1, dim2, dim3]), 0)
        #alpha = np.ones(3)/3

    print("lag", time_lags)

    ### LATC ###
    #print("LATC", seed)
    #c = 10
    #theta = 25
    #rho = 1e-5
    #lambda0 = c * rho

    #tensor_hat, time_latc, mape_latc, rmse_latc = latc(dense_tensor, sparse_tensor, time_lags, 
    #                                                   alpha, rho, lambda0, theta)
    #print("MAPE:", mape_latc*100)
    #print("RMAE:", rmse_latc)
    #mape5_latc.append(mape_latc)
    #rmse5_latc.append(rmse_latc)

    ### THOC ###
    #print("THOC", seed)
    c = 10
    rho = 1e-5
    lambda0 = c * rho
    theta_hoc = 0.03
    
    rhofac = 1.05
    Xest, time_hoc, mape_hoc, rmse_hoc = latc_hoc(dense_tensor, sparse_tensor, time_lags, 
                                                  alpha, rho, lambda0, theta_hoc, rhofac, epsilon=1e-4)
    print("MAPE:", mape_hoc*100)
    print("RMAE:", rmse_hoc)
    mape5_hoc.append(mape_hoc)
    rmse5_hoc.append(rmse_hoc)
    
    ### LATC-HOP  ###
    #print("THOP", seed)
    c = 10
    rho = 1e-5
    lambda0 = c * rho
    theta_hop = 0.03
    p_hop = 0.2
    
    rhofac = 1.05
    Xest, time_hop, mape_hop, rmse_hop = latc_hop(dense_tensor, sparse_tensor, time_lags, 
                                                   alpha, rho, lambda0, theta_hop, rhofac, p_hop, epsilon=1e-4)
    print("mape", mape_hop*100)
    print("rmse", rmse_hop)
    mape5_hop3.append(mape_hop)
    rmse5_hop3.append(rmse_hop)

end_time = time.time()
print(end_time-start_time)

start
missing pattern BM missing rate = 0.3
lag [1 2 3 4 5 6]
iter 20 tole = 0.00191
iter 40 tole = 0.00025
mape 9.098383024218483
rmse 22.59040850937538
lag [1 2 3 4 5 6]
iter 20 tole = 0.00196
iter 40 tole = 0.00026
mape 9.280968572837143
rmse 22.502880021255958
lag [1 2 3 4 5 6]
iter 20 tole = 0.00192
iter 40 tole = 0.00024
mape 9.027498276477433
rmse 22.670290679675237
lag [1 2 3 4 5 6]
iter 20 tole = 0.00185
iter 40 tole = 0.00024
mape 9.17414155601705
rmse 22.55253313736204
lag [1 2 3 4 5 6]
iter 20 tole = 0.00185
iter 40 tole = 0.00024
mape 8.995212035962407
rmse 22.047107174258443
1632.7403428554535


In [67]:
print("MAPE(%) comparison")
#print("LATC:", format(np.mean(mape5_latc)*100,".2f"))
print("LATC-HOC:", format(np.mean(mape5_hoc)*100,".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(mape5_hop)*100,".2f"))
print("--------")
print("RMSE comparison")
#print("LATC:", format(np.mean(rmse5_latc),".2f"))
print("LATC-HOC:", format(np.mean(rmse5_hoc),".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(rmse5_hop),".2f"))

MAPE(%) comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 9.12
LATC-HOP(p=0.6): nan
--------
RMSE comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 22.47
LATC-HOP(p=0.6): nan


### BM 70%

In [70]:
import time
import numpy as np
import scipy.io

time_lags = np.arange(1,7)
alpha = np.ones(3) / 3

#### Main simulations ####

#mape5_latc = []
#rmse5_latc = []

mape5_hoc = []
rmse5_hoc = []

mape5_hop = []
rmse5_hop = []

# missing rates and pattern
missing_rate = 0.7
missing_case = "NM"

print("start")
print("missing pattern", missing_case, "missing rate =", missing_rate)

start_time = time.time()
for seed in range(1,6):
    ### generate missing entries ###
    np.random.seed(seed+999)
    
    if missing_case == 'RM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)
        #alpha = [0.1,0.8,0.1]
    elif missing_case == 'NM':
        sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[2]) + 0.5 - missing_rate)[:, None, :]
        #alpha = np.ones(3)/3
    else:
        dim_time = dim2 * dim3
        block_window = 4
        vec = np.random.rand(int(dim_time / block_window))
        temp = np.array([vec] * block_window)
        vec = temp.reshape([dim2 * dim3], order = 'F')
        sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], 
                                np.array([dim1, dim2, dim3]), 0)
        #alpha = np.ones(3)/3

    print("lag", time_lags)

    ### LATC ###
    #print("LATC", seed)
    #c = 10
    #theta = 25
    #rho = 1e-5
    #lambda0 = c * rho

    #tensor_hat, time_latc, mape_latc, rmse_latc = latc(dense_tensor, sparse_tensor, time_lags, 
    #                                                   alpha, rho, lambda0, theta)
    #print("MAPE:", mape_latc*100)
    #print("RMAE:", rmse_latc)
    #mape5_latc.append(mape_latc)
    #rmse5_latc.append(rmse_latc)

    ### THOC ###
    #print("THOC", seed)
    c = 10
    rho = 1e-5
    lambda0 = c * rho
    theta_hoc = 0.03
    
    rhofac = 1.05
    Xest, time_hoc, mape_hoc, rmse_hoc = latc_hoc(dense_tensor, sparse_tensor, time_lags, 
                                                  alpha, rho, lambda0, theta_hoc, rhofac, epsilon=1e-4)
    print("MAPE:", mape_hoc*100)
    print("RMAE:", rmse_hoc)
    mape5_hoc.append(mape_hoc)
    rmse5_hoc.append(rmse_hoc)
    
    ### LATC-HOP  ###
    #print("THOP", seed)
    c = 10
    rho = 1e-5
    lambda0 = c * rho
    theta_hop = 0.03
    p_hop = 0.2
    
    rhofac = 1.05
    Xest, time_hop, mape_hop, rmse_hop = latc_hop(dense_tensor, sparse_tensor, time_lags, 
                                                   alpha, rho, lambda0, theta_hop, rhofac, p_hop, epsilon=1e-4)
    print("mape", mape_hop*100)
    print("rmse", rmse_hop)
    mape5_hop3.append(mape_hop)
    rmse5_hop3.append(rmse_hop)

end_time = time.time()
print(end_time-start_time)

start
missing pattern BM missing rate = 0.7
lag [1 2 3 4 5 6]
LATC-HOP3 1
iter 20 tole = 0.00559
iter 40 tole = 0.00083
lag [1 2 3 4 5 6]
LATC-HOP3 2
iter 20 tole = 0.00528
iter 40 tole = 0.00084
lag [1 2 3 4 5 6]
LATC-HOP3 3
iter 20 tole = 0.00574
iter 40 tole = 0.00105
lag [1 2 3 4 5 6]
LATC-HOP3 4
iter 20 tole = 0.00513
iter 40 tole = 0.00080
lag [1 2 3 4 5 6]
LATC-HOP3 5
iter 20 tole = 0.00538
iter 40 tole = 0.00091
1932.4174537658691


In [74]:
print("MAPE(%) comparison")
#print("LATC:", format(np.mean(mape5_latc)*100,".2f"))
print("LATC-HOC:", format(np.mean(mape5_hoc)*100,".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(mape5_hop)*100,".2f"))
print("--------")
print("RMSE comparison")
#print("LATC:", format(np.mean(rmse5_latc),".2f"))
print("LATC-HOC:", format(np.mean(rmse5_hoc),".2f"))
print("LATC-HOP(p=0.3):", format(np.mean(rmse5_hop),".2f"))

MAPE(%) comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 10.97
LATC-HOP(p=0.6): nan
--------
RMSE comparison
LATC: nan
LATC-HOC: nan
LATC-HOP(p=0.3): 26.91
LATC-HOP(p=0.6): nan
